In [1]:
%run Global.ipynb
%run Helper_Functions.ipynb

In [7]:
train_df_ohe_selected = pd.read_csv(DATASET_DIR + "/train_df_ohe_selected.csv", keep_default_na=False)

X = train_selected_df.iloc[:,:-1] # (transformed) attributes
y = train_selected_df.iloc[:,-1]  # ground truth labels

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm

# gaussian NB
gnb = GaussianNB()

# linear svm
linearSVM = svm.LinearSVC(random_state=30027, C=1)

# logr
logr = LogisticRegression(solver='sag', max_iter=100)

# decision tree
dt = DecisionTreeClassifier(max_depth = 200, random_state = 30027)

# knn
knn = KNeighborsClassifier(n_neighbors=1)


In [9]:
from sklearn.ensemble import StackingClassifier
estimators = [('gaussianNB', gnb), ('linearSVM', linearSVM), ('logisticRegression', logr), ('decisionTree', dt), ('kNN', knn)]
stacking = StackingClassifier(estimators = estimators, cv = 5)
stacking.fit(X, y)

StackingClassifier(cv=5,
                   estimators=[('gaussianNB', GaussianNB()),
                               ('linearSVM',
                                LinearSVC(C=1, random_state=30027)),
                               ('logisticRegression',
                                LogisticRegression(solver='sag')),
                               ('decisionTree',
                                DecisionTreeClassifier(max_depth=200,
                                                       random_state=30027)),
                               ('kNN', KNeighborsClassifier(n_neighbors=1))])

In [ ]:
cv_evaluate(stacking, X, y)

In [ ]:
stacking.get_params(deep=True)

In [ ]:
param_grid = {'linearSVM__C': range(1,11,5), 'decisionTree__max_depth': range(200, 401, 50), 'kNN__n_neighbors': range(1,6)}
tune_hyperparameter(stacking, param_grid, X, y, cv = 2)